In [62]:
#Импортирование необходимых библиотек
import os
import torch
import re
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [59]:
#Датасет для формирования текста
class WordDataset(data.Dataset):
    def __init__(self, path, prev_chars=3):
        self.prev_chars = prev_chars
        with open(path, "r", encoding="utf8") as file:
            self.text = " ".join((" ".join([i.strip("\ufeff\n.,:;!?-—") for i in file.read().split()]).lower()).split())
            
        self.alphabet = sorted(set(self.text))
        self.alpha_to_int = {alpha:index for index, alpha in enumerate(self.alphabet)}
        self.int_to_alpha = {b:a for a, b in self.alpha_to_int.items()}
        self.len_alphabet = len(self.alphabet)
        self.ohe_vector = torch.eye(self.len_alphabet)
        self.len_text = len(self.text)

    def __getitem__(self, item):
        data = torch.vstack([self.ohe_vector[self.alpha_to_int[self.text[index]]] for index in range(item, item+self.prev_chars)])
        target = self.alpha_to_int[self.text[item+self.prev_chars]]
        return data, target

    def __len__(self):
        return self.len_text - 1 - self.prev_chars

In [67]:
#Класс RNN
class WordRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
        self.layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        y, h = self.rnn(x)
        return self.layer(h)

In [76]:
#Сформируем процесс обучения реккурентной нейронной сети на предсказании символов
eph = 200 #Количество эпох обучения

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

d_train =  WordDataset("train_data_true.txt", prev_chars=10)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=False)

model = WordRNN(input_size=d_train.len_alphabet, hidden_size=64, output_size=d_train.len_alphabet)
model.train()
model.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.00001)

for _e in range(eph):
    lm_count, loss_mean = 0, 0
    train_data_tqdm = tqdm(train_data, leave=True)

    for x_train, y_train in train_data_tqdm:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        predict = model(x_train).squeeze(0) #Убираем первую ось, так как возвращаем h
        loss = loss_func(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1 / lm_count * loss.item() + (1 - 1 / lm_count) * loss_mean

        train_data_tqdm.set_description(f"Текущая эпоха: [{_e+1}/{eph}]  Средняя ошибка: {round(loss_mean, 4)}")

Текущая эпоха: [200/200]  Средняя ошибка: 0.5578: 100%|█████████████████████████████| 202/202 [00:01<00:00, 193.53it/s]


In [77]:
#Сохраним модель
state_dict = model.state_dict()
torch.save(state_dict, "model_rnn1.tar")

In [131]:
#Дообучим данную модель 
eph = 100

d_train =  WordDataset("train_data_true.txt", prev_chars=10)
train_data = data.DataLoader(d_train, batch_size=32, shuffle=False)


model = WordRNN(input_size=d_train.len_alphabet, hidden_size=64, output_size=d_train.len_alphabet)
state_dict = torch.load("model_rnn1.tar", weights_only=True)
model.load_state_dict(state_dict)
model.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.0005, weight_decay=0.000005)

for _e in range(eph):
    lm_count, loss_mean = 0, 0
    train_data_tqdm = tqdm(train_data, leave=True)

    for x_train, y_train in train_data_tqdm:
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        predict = model(x_train).squeeze(0) #Убираем первую ось, так как возвращаем h
        loss = loss_func(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1 / lm_count * loss.item() + (1 - 1 / lm_count) * loss_mean

        train_data_tqdm.set_description(f"Текущая эпоха: [{_e+1}/{eph}]  Средняя ошибка: {round(loss_mean, 4)}")

Текущая эпоха: [100/100]  Средняя ошибка: 0.2887: 100%|█████████████████████████████| 202/202 [00:01<00:00, 201.72it/s]


In [146]:
#Сохраним дообученную нейронную сеть
state_dict = model.state_dict()
torch.save(state_dict, "model_rnn2.tar")

### Результат
Хоть модель явно получилась переобученной, но нам главное проверить качество работы модели

In [147]:
#Протестируем качество работы модели
model = WordRNN(input_size=d_train.len_alphabet, hidden_size=64, output_size=d_train.len_alphabet)
torch.load("model_rnn2.tar", weights_only=True)
model.load_state_dict(state_dict)
model.eval()
model.to("cpu")
predict = "Вы сможете сделать".lower()
total = 150 #Добавление числа символов к итеговой композиции
prev_chars = 10
for _ in range(total):
    train = torch.vstack([d_train.ohe_vector[d_train.alpha_to_int[d_train.text[index]]] for index in range(len(predict)-prev_chars, len(predict))])
    pred = torch.softmax(model(train.unsqueeze(0)).squeeze(0), dim=1)
    predict += d_train.int_to_alpha[torch.argmax(pred).item()]
print(predict)

вы сможете сделатьви мне нравится работа это шедевр изобразительного испуснт а думайте позитивно и верьте в свою способности достигать птричных сузальтатов вы пучший от
